In [1]:
import cv2
import numpy as np
import math

# === Mediapipe ===
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# === Audio Control (PyCaw) ===
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# === Brightness Control ===
import screen_brightness_control as sbc


In [2]:
# Load hand landmark model
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=1)
detector = vision.HandLandmarker.create_from_options(options)

mp_hands = mp.solutions.hands
HAND_CONNECTIONS = mp.solutions.hands.HAND_CONNECTIONS

In [3]:
# Setup Audio Endpoint (PyCaw)
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume_interface = cast(interface, POINTER(IAudioEndpointVolume))

def set_volume_from_distance(dist):
    MIN_DIST, MAX_DIST = 30, 200
    volume = np.interp(dist, [MIN_DIST, MAX_DIST], [0.0, 1.0])  # 0.0 - 1.0
    volume_interface.SetMasterVolumeLevelScalar(volume, None)
    print(f"🔊 Volume set to: {int(volume*100)}%")

def set_brightness_from_distance(dist):
    MIN_DIST, MAX_DIST = 20, 150
    brightness = int(np.interp(dist, [MIN_DIST, MAX_DIST], [0, 100]))  # 0 - 100
    sbc.set_brightness(brightness)
    print(f"💡 Brightness set to: {brightness}%")


In [4]:
def distance(p1, p2):
    """Hitung jarak Euclidean"""
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])

def get_coords(hand_landmarks, image_w, image_h):
    """Ambil koordinat landmark jadi (x,y) integer"""
    return [(int(lm.x * image_w), int(lm.y * image_h)) for lm in hand_landmarks]


In [5]:
mp_hands = mp.solutions.hands
HAND_CONNECTIONS = mp.solutions.hands.HAND_CONNECTIONS

def draw_and_control(image, hand_landmarks):
    image_h, image_w, _ = image.shape
    annotated = image.copy()
    coords = get_coords(hand_landmarks, image_w, image_h)

    # Draw skeleton
    for connection in HAND_CONNECTIONS:
        start = coords[connection[0]]
        end = coords[connection[1]]
        cv2.line(annotated, start, end, (0, 0, 255), 2)
    for (x, y) in coords:
        cv2.circle(annotated, (x, y), 5, (0, 255, 0), -1)

    # === Kontrol Volume (Thumb–Index) ===
    thumb_tip = coords[4]
    index_tip = coords[8]
    dist_vol = distance(thumb_tip, index_tip)
    cv2.line(annotated, thumb_tip, index_tip, (255, 255, 0), 3)
    cv2.putText(annotated, f"Vol:{int(dist_vol)}", (thumb_tip[0], thumb_tip[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)
    set_volume_from_distance(dist_vol)

    # === Kontrol Brightness (Index–Middle) ===
    middle_tip = coords[12]
    dist_bri = distance(index_tip, middle_tip)
    cv2.line(annotated, index_tip, middle_tip, (0, 255, 255), 3)
    cv2.putText(annotated, f"Bri:{int(dist_bri)}", (index_tip[0], index_tip[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)
    set_brightness_from_distance(dist_bri)

    return annotated


In [6]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    result = detector.detect(mp_image)

    annotated_frame = frame.copy()
    for hand_landmarks in result.hand_landmarks:
        annotated_frame = draw_and_control(annotated_frame, hand_landmarks)

    cv2.imshow("Gesture Control: Volume & Brightness", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🔊 Volume set to: 43%
💡 Brightness set to: 11%
🔊 Volume set to: 31%
💡 Brightness set to: 16%
🔊 Volume set to: 31%
💡 Brightness set to: 17%
🔊 Volume set to: 31%
💡 Brightness set to: 16%
🔊 Volume set to: 31%
💡 Brightness set to: 16%
🔊 Volume set to: 31%
💡 Brightness set to: 15%
🔊 Volume set to: 31%
💡 Brightness set to: 16%
🔊 Volume set to: 31%
💡 Brightness set to: 17%
🔊 Volume set to: 30%
💡 Brightness set to: 14%
🔊 Volume set to: 30%
💡 Brightness set to: 52%
🔊 Volume set to: 22%
💡 Brightness set to: 73%
🔊 Volume set to: 18%
💡 Brightness set to: 75%
🔊 Volume set to: 18%
💡 Brightness set to: 76%
🔊 Volume set to: 18%
💡 Brightness set to: 79%
🔊 Volume set to: 19%
💡 Brightness set to: 77%
🔊 Volume set to: 17%
💡 Brightness set to: 78%
🔊 Volume set to: 15%
💡 Brightness set to: 75%
🔊 Volume set to: 14%
💡 Brightness set to: 74%
🔊 Volume set to: 0%
💡 Brightness set to: 3%
🔊 Volume set to: 6%
💡 Brightness set to: 19%
🔊 Volume set to: 5%
💡 Brightness set to: 21%
🔊 Volume set to: 11%
💡 Brightness set 